# This notebook will be used for creating features from our collection of ESG reports, and publicly available data on market capitalisation 


## Approach taken:

Respective sentiment scoring was calculated for each components of ESG using VADER's textual sentiment analysis package. To derive the sentiment score for each sub-pillar, the keywords that best represent each sub-pillar will be used to filter out the sentences that contain those keywords from the entire company's report. The sentences will be iterated through the function to generate a sentiment score, following which an aggregation will be made to generate the average sentiment score for a particular sub-pillar in the company's report.


To retrieve the market capitalisation values, we will be using Yahoo Finance api to send http requests for each company in our dataset. Following the retrieval of each market cap, currency conversion will be done to ensure that all market cap values are valued in the same denomination, before normalising the data.

In [ ]:
import glob
import re
import os
import fitz
import numpy as np
import pandas as pd
import math
import json
import pprint
import gensim
import collections
import spacy
import gensim.corpora as corpora
import nltk
import tqdm


from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.utils import simple_preprocess

# nltk.download('stopwords')
# nltk.download('vader_lexicon')

In [ ]:
# --------------------------- Read a pdf into a large string of text ---------------------------
def read_pdf(file_path):
    pymupdf_text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            pymupdf_text += page.get_text()
    return pymupdf_text


# --------------------------- Read a report and breaks it up into individual sentences ---------------------------
def convert_pdf_into_sentences(text):
    # Remove unnecessary spaces and line breaks
    text = re.sub(r'\x0c\x0c|\x0c', "", str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('\n', ' ', str(text))
    text = ' '.join(text.split())
    text = " " + text + "  "
    text = text.replace("\n", " ")
    if "”" in text: text = text.replace(".”", "”.")
    if "\"" in text: text = text.replace(".\"", "\".")
    if "!" in text: text = text.replace("!\"", "\"!")
    if "?" in text: text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]

    # Filter for sentences with more than 100 characters
    sentences = [s.strip() for s in sentences if len(s) > 100]
    return sentences


# --------------------------- Retrieve the report name from the pdf ---------------------------
def reportName(path):
    name = path.split('/')[-1]
    company = name.split('.')[0]
    return company

Read the entire corpus of ESG/sustainability reports into a python dictionary, where the key - value pairs are of the following:

    - key: Company name
    - value: List containing all the sentences located in the report

In [ ]:
# Read our database of ESG reports
path = 'Reports 2.0'
esg_reports = glob.glob(path + '/*.pdf')
esg_corpus = {}
for report in tqdm.tqdm(esg_reports):
    esg_corpus[reportName(report)] = convert_pdf_into_sentences(read_pdf(report))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 468/468 [02:13<00:00,  3.49it/s]


In [ ]:
# Read the key words from our json file
f = open('keywords.json')
keywordBank = json.load(f)
f.close()

### The following features will be extracted out:

    - Company name
    - ESG Risk Score
    - ESG Risk Rating
    - Opportunities in Clean Tech Sentiment Score
    - Carbon Emissions Sentiment Score
    - Water Stress Sentiment Score
    - Electronic Waste Sentiment Score
    - Toxic Emissions & Waste Sentiment Score	
    - Human Capital Development Sentiment Score	
    - Privacy and Data Security Sentiment Score
    - Labor Management Sentiment Score	
    - Governance Sentiment Score	
    - Market Capitalisation	
    - Page Count	
    - Sentence Count	
    - Word Count

In [ ]:
# --------------------------- Sentiment Analysis ---------------------------
# This function calculates the sentiment score for the various sentences using VADER
# Sentence:
#   - The sentence to be inputted to the function, which will return the respective sentiment score
#   - If there are > 1 sentence, the average will be computed and returned
def averagedCompoundSentimentScore(sentences):
    sid = SentimentIntensityAnalyzer()
    score = 0
    for sentence in sentences:
        sentiment = sid.polarity_scores(sentence)
        score += sentiment['compound']
    try:
        return score / len(sentences)
    except ZeroDivisionError:
        return score

# --------------------------- Sentence Extraction ---------------------------
# This function extracts out the keywords from the given corpus
# corpus: 
#   - This refers to a document (i.e one company)
# subFeatureKeywords:
#   - A list containing all the keywords which we would like to identify from our sentence bank
def keySentences(corpus, subFeatureKeywords):
    sentencesCaptured = []
    for word in subFeatureKeywords:
        sentencesCaptured.extend([sentence for sentence in corpus if word in sentence])
    return sentencesCaptured

# --------------------------- Print all sentences (Debugging purposes only) ---------------------------
def printAllSentences(corpus, pillar, keywordBank):
    for subFeature, kewords in keywordBank[pillar].items():
        print('\n\n\n')
        print(f"======= Printing Sentences from: '{subFeature}' =======")
        sentences = keySentences(corpus, kewords)
        for sentence in sentences:
            print(sentence)
            print('\n\n')
        
# --------------------------- Subpillar Feature Statistics ---------------------------
# 4 options for pillar: 
#   - 'Environment'
#   - 'Social'
#   - 'Governance'
#   - 'ESG phrases'
# corpus: 
#   - A specific company report, and NOT the whole collection of reports from all companies!
# keywordBank: 
#   - All the keywords from the subpillar
def subPillar_featureStats(corpus, pillar, keywordBank):
    data = {}

    # Calculate the sentences, frequency of sentence occurence, sentiment score etc
    def summaryStatistics(corpus, subFeatureKeywords):
        temp = {
            # "Sentences": None,
            "NumOfSentences": None,
            "FrequencyOfOccurence": None,
            "SentimentScore": None
        }
        # temp['Sentences'] = subpillar_sentences(corpus, keywordBank[pillar])
        sentences = keySentences(corpus, subFeatureKeywords)
        temp['NumOfSentences'] = len(sentences)
        temp['FrequencyOfOccurence'] = round(len(sentences) / len(corpus), 5)
        temp['SentimentScore'] = averagedCompoundSentimentScore(sentences)
        return temp

    for subFeature, subFeatureKeywords in keywordBank[pillar].items():
        data[subFeature] = summaryStatistics(corpus, subFeatureKeywords)
    
    return data

# --------------------------- Complete Feature Statistics ---------------------------
# This combines all the data across the 3 pillars into a dictionary 
# esg_bank:
#   - Complete set of data processed from reading in all the companies
#   - Structure of esg_bank:
#       - Dictionary where
#           - key: company name
#           - value: [sentence1, sentence2, ..., sentenceN]
# companyName:
#   - The company we wish to explore
# keywordBank:
#   - Complete set of data from the keywords.json file
def featureStats(esg_bank, companyName, keywordBank):
    company = {
        companyName: []
    }
    for pillar in [*keywordBank][:-1]:
        temp = {}
        temp[pillar] = subPillar_featureStats(esg_bank[companyName], pillar, keywordBank)
        company[companyName].append(temp)
    return company
    
def processByLength(esg_bank, keywordBank, numberOfReports):
    print('\n\n === Generating feature statistic data from all companies === \n\n')
    companies = []

    def flatten_data(dictionary_data):
        new_data = {
            "Companies": list(dictionary_data.keys())[0],
        }
        for subData in dictionary_data.values():
            for i in range(0, 3):
                for pillar, pillarValues in subData[i].items():
                    for title, data in pillarValues.items():
                        new_data[title + ' Sentiment Score'] = data['SentimentScore'] 
        return new_data

    counter = 0
    for company, data in tqdm.tqdm(esg_bank.items()):
        if counter == numberOfReports:
            break
        else:
            company_data = featureStats(esg_bank, company, keywordBank)
            companies.append(flatten_data(company_data))
            counter += 1

    return pd.DataFrame(companies)

In [ ]:
company_scores = processByLength(esg_corpus, keywordBank, len(esg_corpus))



 === Generating feature statistic data from all companies === 




100%|███████████████████████████████████████████████████████████████████████████████████████████| 468/468 [02:52<00:00,  2.72it/s]


In [ ]:
print(f"Length of dataframe: {len(company_scores)}")
print("Print the first 5 rows")
company_scores.head(5)

Length of dataframe: 468
Print the first 5 rows


,Companies,Opportunities in Clean Tech Sentiment Score,Carbon Emissions Sentiment Score,Water Stress Sentiment Score,Electronic Waste Sentiment Score,Toxic Emissions & Waste Sentiment Score,Human Capital Development Sentiment Score,Privacy and Data Security Sentiment Score,Labor Management Sentiment Score,Governance Sentiment Score
0,Enea,0.425327,0.557400,0.072240,0.234367,0.000000,0.375837,0.355967,0.327428,0.284460
1,"TIS, Inc",0.521842,0.432322,0.996200,0.239570,0.000000,0.535148,0.416305,0.501125,0.564920
2,Sopra Steria Group,0.504519,0.281237,0.347221,0.022540,0.144443,0.452177,0.392583,0.459815,0.456827
3,"Rackspace Technology Global, Inc",0.544586,0.101990,0.229163,0.006082,0.542300,0.213250,0.465908,0.532997,0.347849
4,Infomedia Ltd,0.341683,-0.401900,0.000000,0.547075,0.000000,0.483650,0.396403,0.375560,0.378634


## The market capitalisation and ESG risk ratings data was processed from another jupyter notebook. As such, we will import the processed data.

In [ ]:
# Read in data on ESG risk ratings
corporate = pd.read_csv('Corporate2.0.csv')

# Filter out those that contains the report
corporate_new = corporate.loc[corporate['Annual Report'].notnull()].reset_index(drop=True)

# Select relevant columns
corporate_new = corporate_new[['Companies', 'ESG Risk Score', 'ESG Risk Rating']]

In [ ]:
# Read data on market cap
mkt_cap = pd.read_csv('mktcap.csv')

# Select the relevant columns
mkt_cap = mkt_cap[[
    'S/N', 'Companies', 'Market Capitalisation', 
    'Currency', 'Ticker', 'Note if Currency is not USD', 
    'mktcapfunction'
]]

# Filter out valid market cap
def valid(value):
    if math.isnan(value) or value == 0:
        return 0
    else:
        return 1

mkt_cap['Valid'] = mkt_cap['mktcapfunction'].apply(lambda x: valid(x))
valid_mkt_cap = mkt_cap[mkt_cap['Valid'] == 1].reset_index(drop=True)

valid_mkt_cap = valid_mkt_cap[['Companies', 'mktcapfunction']]
valid_mkt_cap.rename({'mktcapfunction': 'Market Capitalisation'}, axis=1, inplace=True)

Now that we have processed our dataframe for company's ESG risk ratings as well as the market cap, we will merge it with our main dataframe on ESG features

In [ ]:
# Merge with ESG risk ratings dataframe
corporate_new_merged = pd.merge(corporate_new, company_scores, how='inner', on='Companies').reset_index(drop=True)

# Merge with Market Cap dataframe
final = pd.merge(corporate_new_merged, valid_mkt_cap, how='inner', on='Companies').reset_index(drop=True)

In [ ]:
print(f"Length of dataframe: {len(final)}")
print("Print the first 5 rows")
final.head(5)

Length of dataframe: 434
Print the first 5 rows


,Companies,ESG Risk Score,ESG Risk Rating,Opportunities in Clean Tech Sentiment Score,Carbon Emissions Sentiment Score,Water Stress Sentiment Score,Electronic Waste Sentiment Score,Toxic Emissions & Waste Sentiment Score,Human Capital Development Sentiment Score,Privacy and Data Security Sentiment Score,Labor Management Sentiment Score,Governance Sentiment Score,Market Capitalisation
0,24SevenOffice Group,24.6,Medium ESG Risk,0.357958,0.000000,0.00000,0.000000,0.0,0.471713,0.375589,0.218405,0.330786,8.192850e+07
1,"2U, Inc",19.8,Low ESG Risk,0.226180,-0.077200,-0.07720,-0.077200,0.0,0.563257,0.406256,0.275578,0.390339,9.757620e+08
2,Absolute Software Corp,24.1,Medium ESG Risk,-0.279050,-0.229400,-0.42150,-0.421500,0.0,0.000000,0.231411,0.827100,0.488943,4.344337e+08
3,Accenture,9.7,Negligible ESG Risk,0.632968,0.453108,-0.42150,0.744725,0.0,0.610265,0.381267,0.518605,0.600880,2.160000e+11
4,"ACI Worldwide, Inc",20.8,Medium ESG Risk,0.399900,0.353917,0.47024,0.177225,0.0,0.678200,0.290737,0.232244,0.263880,3.149302e+09


In [ ]:
# Check which columns do not match
coy_sorted = company_scores.sort_values(['Companies'])
coy_sorted = coy_sorted[['Companies']].reset_index(drop=True)

corporate_new_sorted = corporate_new.sort_values(['Companies'])
corporate_new_sorted = corporate_new_sorted[['Companies']].reset_index(drop=True)

count = 0
for company in list(coy_sorted['Companies']):
    if company not in list(corporate_new_sorted['Companies']):
        print(f"Company not found: {company}")
        count += 1
print(f"{count} discrepancy/discrepancies found!")

Company not found: NASDAQ_HCAT_2019
1 discrepancy/discrepancies found!


In [ ]:
# Create a new column for the number of pages per report
def page_count(file_path):
    with fitz.open(file_path) as doc:
        return len(doc)

# ESG Corpus
path = 'Reports 2.0'
esg_reports = glob.glob(path + '/*.pdf')
report_pages = {}
for report in esg_reports:
    report_pages[reportName(report)] = page_count(report)

# Create a new column for the number of sentences per report
company_sentences = {}
for company, sentences in esg_corpus.items():
    company_sentences[company] = len(sentences)
    

# Create a new column that tracks the number of words in each report
words = {}
for company, sentences in esg_corpus.items():
    sentences_combined = ' '.join(sentences)
    words[company] = len(sentences_combined.split(' '))
    
final['Page Count'] = final['Companies'].apply(lambda x: report_pages[x])
final['Sentence Count'] = final['Companies'].apply(lambda x: company_sentences[x])
final['Word Count'] = final['Companies'].apply(lambda x: words[x])

In [ ]:
print(f"Length of dataframe: {len(final)}")
print("Print the first 5 rows")
final.head(5)

Length of dataframe: 434
Print the first 5 rows


,Companies,ESG Risk Score,ESG Risk Rating,Opportunities in Clean Tech Sentiment Score,Carbon Emissions Sentiment Score,Water Stress Sentiment Score,Electronic Waste Sentiment Score,Toxic Emissions & Waste Sentiment Score,Human Capital Development Sentiment Score,Privacy and Data Security Sentiment Score,Labor Management Sentiment Score,Governance Sentiment Score,Market Capitalisation,Page Count,Sentence Count,Word Count
0,24SevenOffice Group,24.6,Medium ESG Risk,0.357958,0.000000,0.00000,0.000000,0.0,0.471713,0.375589,0.218405,0.330786,8.192850e+07,44,251,8575
1,"2U, Inc",19.8,Low ESG Risk,0.226180,-0.077200,-0.07720,-0.077200,0.0,0.563257,0.406256,0.275578,0.390339,9.757620e+08,38,156,5429
2,Absolute Software Corp,24.1,Medium ESG Risk,-0.279050,-0.229400,-0.42150,-0.421500,0.0,0.000000,0.231411,0.827100,0.488943,4.344337e+08,2,19,474
3,Accenture,9.7,Negligible ESG Risk,0.632968,0.453108,-0.42150,0.744725,0.0,0.610265,0.381267,0.518605,0.600880,2.160000e+11,15,150,4690
4,"ACI Worldwide, Inc",20.8,Medium ESG Risk,0.399900,0.353917,0.47024,0.177225,0.0,0.678200,0.290737,0.232244,0.263880,3.149302e+09,39,150,4914


In [ ]:
# Output data to csv file
final.to_csv('data.csv')